In [2]:
import numpy as np
from scipy.sparse import csc_matrix

# prepare data

row = np.array([0, 2, 2, 0, 1, 2, 0])
col = np.array([0, 0, 1, 2, 2, 2, 0])
data = np.array([1, 2, 3, 4, 5, 6, 5])
sp_A = csc_matrix((data, (row, col)), shape=(3, 3), dtype=float)

In [5]:
from scipy.sparse.linalg import spsolve

b = np.array([4, 5, 6])
x = spsolve(sp_A, b)
print(x)

[0. 0. 1.]


In [258]:
import numpy as np
import trimesh

# read unstructured mesh
uns_mesh: trimesh.Trimesh = trimesh.load_mesh('../../static/unsmesh/N2_modified.obj')
str_mesh = trimesh.Trimesh()

def vert_dist(msh: trimesh.Trimesh, vidx1: int, vidx2: int) -> float:
    return np.linalg.norm(
        msh.vertices[vidx1] - msh.vertices[vidx2]
    )

In [259]:
# get mesh inner and boundary vertices
# output:
#   - inn_verts
#   - bnd_verts
#   - bnd_length
import numpy_indexed as npi

bnd_edges = npi.difference(uns_mesh.edges_unique, uns_mesh.face_adjacency_edges)
bnd_verts = np.array([*bnd_edges[0]])
bnd_edges = np.delete(bnd_edges, [0], axis=0)
bnd_length = vert_dist(uns_mesh, *bnd_verts[:2])

success = True
while success:
    success = False
    last = bnd_verts[-1]
    for idx in range(len(bnd_edges)):
        if last == bnd_edges[idx][0]:
            success = True
            last = bnd_edges[idx][1]
        elif last == bnd_edges[idx][1]:
            success = True
            last = bnd_edges[idx][0]
        if success:
            bnd_verts = np.append(bnd_verts, last)
            bnd_edges = np.delete(bnd_edges, [idx], axis=0)
            bnd_length += vert_dist(uns_mesh, *bnd_verts[-2:])
            break

inn_verts = npi.difference(uns_mesh.face_adjacency_edges.flatten(), bnd_verts)

In [260]:
# parameterize bound to Square
# assume Z=0.0 in str_mesh
# output:
#   - str_mesh.vertices (only boundary)

from functools import reduce

_scale = 2 # square edge length

last_v = bnd_verts[0]
accumed = 0.

for bnd_v in bnd_verts[1:]:
    old_ratio = accumed / bnd_length
    accumed += vert_dist(uns_mesh, last_v, bnd_v)
    ratio = accumed / bnd_length
    flag = -reduce(
        lambda x, y: x * (1 if (y - old_ratio) * (y - ratio) > 0 else -y),
        [0.25, 0.5, 0.75],
        1
    )
    ratio = max(ratio, flag)
    vpos = (0., 0.)
    if ratio < 0.25:
        vpos = (-(_scale / 2) + _scale * (ratio / 0.25), -_scale / 2)
    elif ratio < 0.5:
        vpos = (_scale / 2,  -(_scale / 2) + _scale * ((ratio - 0.25) / 0.25))
    elif ratio < 0.75:
        vpos = ((_scale / 2) - _scale * ((ratio - 0.5) / 0.25), _scale / 2)
    else:
        vpos = (-_scale / 2, (_scale / 2) - _scale * ((ratio - 0.75) / 0.25))

    str_mesh.vertices = np.vstack([str_mesh.vertices, np.append(vpos, 0.)])
    last_v = bnd_v

In [262]:
# initial weights
from scipy.sparse import csc_matrix


[1.59368222e-01 1.29695403e-01 4.18641431e-02 ... 5.36682503e-02
 5.19593470e-06 4.12151977e-03]
[[ 3924  5625]
 [ 1666 10046]
 [    2 14336]
 ...
 [16934 16935]
 [ 3160 16935]
 [ 5768 16934]]
